In [185]:
import os
import logging

import traceback

from datetime import datetime, time
from os import access

from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.types import (StructType, StructField, IntegerType, StringType, DateType, LongType)
from pyspark.sql import functions as F

Define file paths

In [186]:
current_dir = os.getcwd()
historical_attendance_2006_2015_file_names = '../csv_files/chapter15/nyc_school_attendance/*Historical_DateFormat1*.csv'
historical_attendance_2015_2018_file_names = '../csv_files/chapter15/nyc_school_attendance/*Historical_DateFormat2*.csv'
daily_attendance_file_name = '../csv_files/chapter15/nyc_school_attendance/2018-2019_Daily_Attendance.csv'

historical_attendance_2006_2015_file_paths = os.path.join(current_dir, historical_attendance_2006_2015_file_names)
historical_attendance_2015_2018_file_paths = os.path.join(current_dir, historical_attendance_2015_2018_file_names)
daily_attendance_file_path = os.path.join(current_dir, daily_attendance_file_name)

Configure logging

In [187]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

Methods

In [188]:
def ingest_historical_attendance(spark: SparkSession, date_format: str, file_path: str):
    schema = StructType([StructField('School', StringType(), False),
                         StructField('date', DateType(), False),
                         StructField('schoolYear', StringType(), False),
                         StructField('enrolled', IntegerType(), False),
                         StructField('present', IntegerType(), False),
                         StructField('absent', IntegerType(), False),
                         StructField('released', IntegerType(), False)])
    df = (spark.read.format("csv").option("header", True)
          .option("dateFormat", date_format)
          .schema(schema)
          .load(file_path))
    return df


def ingest_daily_attendance_2018(spark: SparkSession, date_format: str, file_path: str):
    schema = StructType([StructField('School DBN', StringType(), False),
                         StructField('date', DateType(), False),
                         StructField('enrolled', IntegerType(), False),
                         StructField('absent', IntegerType(), False),
                         StructField('present', IntegerType(), False),
                         StructField('released', IntegerType(), False)])
    df = (spark.read.format("csv").option("header", True)
          .option("dateFormat", date_format)
          .schema(schema)
          .load(file_path))
    df = df.withColumn("schoolYear", F.lit(2018)).withColumnRenamed("School DBN", "School").drop("School DBN")

    return df

Ingest Data and Do Analysis

In [189]:
spark = SparkSession.builder.appName("NYC School Attendance Analysis").getOrCreate()
spark.sparkContext.setLogLevel("WARN")

Load datasets


In [190]:
historical_attendance_2006_2015_df = ingest_historical_attendance(spark, "yyyyMMdd",
                                                                  historical_attendance_2006_2015_file_paths)
historical_attendance_2015_2018_df = ingest_historical_attendance(spark, "MM/dd/yyyy",
                                                                  historical_attendance_2015_2018_file_paths)
historical_attendance_df = historical_attendance_2006_2015_df.union(historical_attendance_2015_2018_df)
historical_attendance_df.show()

+------+----------+----------+--------+-------+------+--------+
|School|      date|schoolYear|enrolled|present|absent|released|
+------+----------+----------+--------+-------+------+--------+
|01M015|2006-09-05|  20062007|     252|    226|    26|       0|
|01M015|2006-09-06|  20062007|     248|    227|    21|       0|
|01M015|2006-09-07|  20062007|     245|    228|    17|       0|
|01M015|2006-09-08|  20062007|     246|    226|    20|       0|
|01M015|2006-09-11|  20062007|     248|    223|    25|       0|
|01M015|2006-09-12|  20062007|     247|    223|    24|       0|
|01M015|2006-09-13|  20062007|     244|    227|    17|       0|
|01M015|2006-09-14|  20062007|     243|    217|    26|       0|
|01M015|2006-09-15|  20062007|     240|    226|    14|       0|
|01M015|2006-09-18|  20062007|     242|    224|    18|       0|
|01M015|2006-09-19|  20062007|     243|    220|    23|       0|
|01M015|2006-09-20|  20062007|     244|    228|    16|       0|
|01M015|2006-09-21|  20062007|     243| 

In [191]:
daily_attendance_2018_df = ingest_daily_attendance_2018(spark, "yyyyMMdd", daily_attendance_file_path)
daily_attendance_2018_df.show()

+------+----------+--------+------+-------+--------+----------+
|School|      date|enrolled|absent|present|released|schoolYear|
+------+----------+--------+------+-------+--------+----------+
|01M015|2018-09-05|     172|    19|    153|       0|      2018|
|01M015|2018-09-06|     171|    17|    154|       0|      2018|
|01M015|2018-09-07|     172|    14|    158|       0|      2018|
|01M015|2018-09-12|     173|     7|    166|       0|      2018|
|01M015|2018-09-13|     173|     9|    164|       0|      2018|
|01M015|2018-09-14|     173|    11|    162|       0|      2018|
|01M015|2018-09-17|     173|    10|    163|       0|      2018|
|01M015|2018-09-18|     174|     7|    167|       0|      2018|
|01M015|2018-09-20|     174|     7|    167|       0|      2018|
|01M015|2018-09-21|     174|     8|    166|       0|      2018|
|01M015|2018-09-24|     174|    13|    161|       0|      2018|
|01M015|2018-09-25|     174|     9|    165|       0|      2018|
|01M015|2018-09-26|     174|     6|    1

In [192]:
master_df = historical_attendance_df.union(daily_attendance_2018_df).withColumn("schoolYear",
                                                                                F.substring(F.col("schoolYear"), 1, 4))
master_df.show()

+------+----------+----------+--------+-------+------+--------+
|School|      date|schoolYear|enrolled|present|absent|released|
+------+----------+----------+--------+-------+------+--------+
|01M015|2006-09-05|      2006|     252|    226|    26|       0|
|01M015|2006-09-06|      2006|     248|    227|    21|       0|
|01M015|2006-09-07|      2006|     245|    228|    17|       0|
|01M015|2006-09-08|      2006|     246|    226|    20|       0|
|01M015|2006-09-11|      2006|     248|    223|    25|       0|
|01M015|2006-09-12|      2006|     247|    223|    24|       0|
|01M015|2006-09-13|      2006|     244|    227|    17|       0|
|01M015|2006-09-14|      2006|     243|    217|    26|       0|
|01M015|2006-09-15|      2006|     240|    226|    14|       0|
|01M015|2006-09-18|      2006|     242|    224|    18|       0|
|01M015|2006-09-19|      2006|     243|    220|    23|       0|
|01M015|2006-09-20|      2006|     244|    228|    16|       0|
|01M015|2006-09-21|      2006|     243| 

Start to data analysis

In [193]:
# Question: What is the average enrollment for each school?
average_enrollment_df = (master_df.groupBy("School", "schoolYear")
                         .avg("enrolled")
                         .withColumnRenamed("avg(enrolled)", "average_enrollment")
                         .orderBy("average_enrollment", ascending=False))
average_enrollment_df.show()

+------+----------+------------------+
|School|schoolYear|average_enrollment|
+------+----------+------------------+
|13K430|      2017| 5819.798780487805|
|13K430|      2016| 5645.208588957055|
|13K430|      2015| 5512.363636363636|
|13K430|      2013| 5430.193939393939|
|13K430|      2014| 5429.263473053892|
|13K430|      2012|5423.2760736196315|
|13K430|      2011| 5302.305882352941|
|13K430|      2010|5111.8452380952385|
|13K430|      2009|            4913.0|
|13K430|      2008| 4626.764705882353|
|20K490|      2017| 4531.091463414634|
|26Q430|      2017| 4507.579268292683|
|26Q430|      2009| 4439.734939759036|
|26Q430|      2008|  4424.80473372781|
|26Q430|      2016| 4424.475903614458|
|26Q430|      2006| 4418.317647058823|
|20K490|      2016| 4410.257668711656|
|26Q430|      2007| 4407.773255813953|
|13K430|      2007| 4386.145348837209|
|10X440|      2006| 4380.070588235294|
+------+----------+------------------+
only showing top 20 rows



In [194]:
# Evolution of enrollment by year
evolution_of_enrollment_df = (average_enrollment_df.groupBy("schoolYear")
                              .agg(F.sum("average_enrollment").alias("sum_enrollment"))
                              .withColumn("sum_enrollment", F.floor(F.col("sum_enrollment").cast(LongType())))
                              .orderBy("sum_enrollment", ascending=False))
evolution_of_enrollment_df.show()

+----------+--------------+
|schoolYear|sum_enrollment|
+----------+--------------+
|      2006|        994597|
|      2011|        990235|
|      2010|        990084|
|      2009|        987968|
|      2012|        986900|
|      2013|        985040|
|      2014|        983189|
|      2007|        978064|
|      2015|        977576|
|      2008|        976091|
|      2016|        971130|
|      2017|        963861|
|       478|          2086|
|       431|          1888|
|       432|          1871|
|       479|          1820|
|       441|          1787|
|       474|          1781|
|       480|          1771|
|       341|          1731|
+----------+--------------+
only showing top 20 rows



In [195]:
# Delta of enrollment by year
max_enrollment_df = evolution_of_enrollment_df.first()
print(max_enrollment_df)
evolution_of_enrollment_with_delta_df = (
    evolution_of_enrollment_df.withColumn("max", F.lit(max_enrollment_df["sum_enrollment"]))
    .withColumn("delta_enrollment", F.expr("max - sum_enrollment"))
    .drop("max"))
evolution_of_enrollment_with_delta_df.show()

Row(schoolYear='2006', sum_enrollment=994597)


+----------+--------------+----------------+
|schoolYear|sum_enrollment|delta_enrollment|
+----------+--------------+----------------+
|      2006|        994597|               0|
|      2011|        990235|            4362|
|      2010|        990084|            4513|
|      2009|        987968|            6629|
|      2012|        986900|            7697|
|      2013|        985040|            9557|
|      2014|        983189|           11408|
|      2007|        978064|           16533|
|      2015|        977576|           17021|
|      2008|        976091|           18506|
|      2016|        971130|           23467|
|      2017|        963861|           30736|
|       478|          2086|          992511|
|       431|          1888|          992709|
|       432|          1871|          992726|
|       479|          1820|          992777|
|       441|          1787|          992810|
|       474|          1781|          992816|
|       480|          1771|          992826|
|       34

In [196]:
# Question: Maximum enrollment per year of each school
max_enrollment_df = (master_df.groupBy("School", "schoolYear")
                     .agg(F.max("enrolled").alias("max_enrollment"))
                     .orderBy("max_enrollment", ascending=False))
max_enrollment_df.show()

+------+----------+--------------+
|School|schoolYear|max_enrollment|
+------+----------+--------------+
|13K430|      2017|          5850|
|13K430|      2016|          5676|
|13K430|      2015|          5550|
|13K430|      2012|          5497|
|13K430|      2013|          5481|
|13K430|      2014|          5467|
|13K430|      2011|          5353|
|13K430|      2010|          5185|
|13K430|      2009|          4982|
|13K430|      2008|          4685|
|20K490|      2017|          4620|
|26Q430|      2017|          4576|
|10X440|      2006|          4571|
|10X440|      2007|          4542|
|26Q430|      2006|          4542|
|10X440|      2009|          4534|
|26Q430|      2009|          4490|
|10X440|      2010|          4486|
|20K490|      2016|          4484|
|13K430|      2007|          4475|
+------+----------+--------------+
only showing top 20 rows



In [197]:
# Question: Minimum of absence per year of each school
min_absence_df = (master_df.groupBy("School", "schoolYear")
                  .agg(F.min("absent").alias("min_absent"))
                  .orderBy("min_absent"))
min_absence_df.show()

+------+----------+----------+
|School|schoolYear|min_absent|
+------+----------+----------+
|10X284|      2008|         0|
|12X267|      2006|         0|
|11X416|      2006|         0|
|10X414|      2008|         0|
|11X417|      2007|         0|
|06M540|      2007|         0|
|03M243|      2006|         0|
|05M283|      2007|         0|
|07X547|      2008|         0|
|02M439|      2006|         0|
|10X268|      2006|         0|
|13K266|      2007|         0|
|03M208|      2006|         0|
|08X332|      2006|         0|
|02M473|      2007|         0|
|10X430|      2006|         0|
|02M625|      2007|         0|
|12X262|      2006|         0|
|08X518|      2007|         0|
|05M318|      2007|         0|
+------+----------+----------+
only showing top 20 rows



In [199]:
average_absenteeism_percentage_df = (master_df.groupBy("School")
                                     .agg(F.avg("enrolled").alias("average_enrollment"),
                                          F.avg("absent").alias("average_absenteeism"))
                                     .withColumn("average_absenteeism_percentage",
                                                 F.expr("(average_absenteeism / average_enrollment) *  100"))
                                     )

# Question: Top 5 schools with the least absenteeism
logger.info("#Question: Top 5 schools with the least absenteeism")
average_absenteeism_percentage_df.show(5)

# Question: Top 5 schools with the most absenteeism
logger.info("#Question: Top 5 schools with most absenteeism")
average_absenteeism_percentage_df.orderBy("average_absenteeism_percentage", ascending=False).show(5)

2025-01-26 09:48:01,291 - INFO - #Question: Top 5 schools with the least absenteeism
2025-01-26 09:48:02,445 - INFO - #Question: Top 5 schools with most absenteeism 


+------+------------------+-------------------+------------------------------+
|School|average_enrollment|average_absenteeism|average_absenteeism_percentage|
+------+------------------+-------------------+------------------------------+
|02M042| 742.0204359673024|  17.23887375113533|            2.3232343633046475|
|02M529| 661.0445859872611| 125.33414992650661|            18.960014586508073|
|08X093| 386.9034013605442| 31.672562358276643|              8.18616798066396|
|01M064| 283.0816326530612|  25.69342403628118|             9.076330313443716|
|01M539|1489.4512913457181|  60.30357951971001|             4.048711083745866|
+------+------------------+-------------------+------------------------------+
only showing top 5 rows



+------+------------------+-------------------+------------------------------+
|School|average_enrollment|average_absenteeism|average_absenteeism_percentage|
+------+------------------+-------------------+------------------------------+
|10X476|175.64229765013056| 112.70496083550914|             64.16732321505552|
|79Q979| 130.6936416184971|  74.28901734104046|             56.84210526315789|
|79K957|134.52469135802468|  71.67901234567901|            53.283164318817974|
|22K585|207.41176470588235| 106.46323529411765|             51.32941009642656|
|79X921|126.02173913043478|  64.29891304347827|             51.02208038640676|
+------+------------------+-------------------+------------------------------+
only showing top 5 rows

